<a href="https://colab.research.google.com/github/ben900926/Plant-seedling-classification/blob/main/AI_final_project_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt # GRAPHING AND VISUALIZATIONS
import os
import cv2
from sklearn import preprocessing # for data preprocess, e.g label encoding
from sklearn.model_selection import train_test_split # for splitting train data for validation
# for convolutional
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import BatchNormalization
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,CSVLogger
from sklearn.metrics import confusion_matrix
# convolution end
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator


In [5]:
scale = 70
seed = 7

In [26]:
class Image_dataset:

    def __init__(self):
        self.train_set = list()
        self.original_labels = list()
        self.train_labels = list() # store files address for testing
        self.x_train = []
        self.x_val = []
        self.y_train = []
        self.y_val = []

    # load training sets from image files
    def load_train_file(self,path):
        dirs = os.listdir(path) # read images from given file
        for dir in dirs:
            count = 0
            for img in os.listdir(os.path.join(path,dir)):
                img = os.path.join(path,dir,img)
                self.train_set.append(cv2.resize(cv2.imread(img),(scale,scale)))
                self.train_labels.append(dir)
                count += 1
            print(f"{dir}: load {count} images done!")
        self.train_set = np.asarray(self.train_set)
        print('\n')
        print('Orignal xtrain shape : {}'.format(self.train_set.shape)) #(4750, dim, dim, 3)
        self.train_labels = pd.DataFrame(self.train_labels)
    
    # load testing sets from single directory
    def load_test_file(self,path):
      dir = os.listdir(path)
      count = 0
      for img in dir:
        self.train_labels.append(img)
        img = os.path.join(path,img)
        self.train_set.append(cv2.resize(cv2.imread(img),(scale,scale)))
        count += 1
      print(f"load {count} images done!")
      self.train_set = np.asarray(self.train_set)
      print('\n')
      print('Test xtrain shape : {}'.format(self.train_set.shape))
    
    # convert image to hsv, remove background and noise
    def clean_img(self):
        new_train = []
        for i in self.train_set:
            blurr = cv2.GaussianBlur(i,(5,5),0)
            hsv = cv2.cvtColor(blurr,cv2.COLOR_BGR2HSV)
            # GREEN PARAMETERS
            lower = (25,40,50)
            upper = (75,255,255)
            mask = cv2.inRange(hsv,lower,upper)
            struc = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(11,11))
            mask = cv2.morphologyEx(mask,cv2.MORPH_CLOSE,struc)
            boolean = mask>0
            new = np.zeros_like(i,np.uint8)
            new[boolean] = i[boolean]
            new_train.append(new)
        self.train_set = np.asarray(new_train)
    
    # use sklearn to do label encoding
    def label_encoding(self):
        labels = preprocessing.LabelEncoder()
        labels.fit(self.train_labels[0]) # collect label from the datasets of images
        self.original_labels = labels
        # tranform label into binary format
        encoded_label = labels.transform(self.train_labels[0])
        print('\n')
        print('Classes'+str(labels.classes_))
        binary_label = np_utils.to_categorical(encoded_label)
        self.train_labels = binary_label    

    # split the train data to prevent overfitting
    def split_data(self):
      # normalization
      self.train_set = self.train_set/255
      self.x_train, self.x_val, self.y_train, self.y_val = train_test_split(self.train_set, self.train_labels, test_size=0.1, random_state=seed, stratify=self.train_labels)
      print('\n---Split training and validation sets---')
      print('xtrain shape : {}'.format(self.x_train.shape))
      print('ytrain shape : {}'.format(self.y_train.shape))
      print('  xval shape : {}'.format(self.x_val.shape))
      print('  yval shape : {}'.format(self.y_val.shape))
      print('----------------------------------------------')
      # save the datasets
      np.savez('/content/drive/MyDrive/AI_final/plant_split_dataset.npz',
               train_set=self.train_set,train_labels=self.train_labels,x_train=self.x_train,x_val=self.x_val,y_train=self.y_train,y_val=self.y_val)


In [15]:
class convolutional_network:

  def __init__(self, epoches, batch_size):
    dataset = np.load('/content/drive/MyDrive/AI_final/plant_split_dataset.npz') # load dataset from file
    self.train_set = dataset['train_set']
    self.train_labels = dataset['train_labels']
    self.x_train = dataset['x_train']
    self.x_val = dataset['x_val']
    self.y_train = dataset['y_train']
    self.y_val = dataset['y_val']
    #self.datagen = None
    # for training
    self.epoches = epoches
    self.batch_size = batch_size
    # define neural network
    self.model = Sequential()
  # construct neural network structure
  def network_layer(self):
    np.random.seed(seed)
    # add layers : 4 convolution layers and 3 fully connected layers
    self.model.add(Conv2D(filters=64, kernel_size=(5, 5), input_shape=(scale, scale, 3), activation='relu'))
    self.model.add(BatchNormalization(axis=3))
    # convolution + pooling layer
    self.model.add(Conv2D(filters=64, kernel_size=(5, 5), activation='relu'))
    self.model.add(MaxPooling2D((2, 2))) # resist noise and reduce computation resources
    self.model.add(BatchNormalization(axis=3))
    self.model.add(Dropout(0.1)) # ignored 0.1 neural to prevent overfitting
    self.model.add(Conv2D(filters=128, kernel_size=(5, 5), activation='relu'))
    self.model.add(BatchNormalization(axis=3))
    self.model.add(Conv2D(filters=128, kernel_size=(5, 5), activation='relu'))
    self.model.add(MaxPooling2D((2, 2)))
    self.model.add(BatchNormalization(axis=3))
    self.model.add(Dropout(0.1))
    self.model.add(Conv2D(filters=256, kernel_size=(5, 5), activation='relu'))
    self.model.add(BatchNormalization(axis=3))
    self.model.add(Conv2D(filters=256, kernel_size=(5, 5), activation='relu'))
    self.model.add(MaxPooling2D((2, 2)))
    self.model.add(BatchNormalization(axis=3))
    self.model.add(Dropout(0.1))
    self.model.add(Flatten()) # flatten map to a single vector
    self.model.add(Dense(256, activation='relu')) # fully connected networks
    self.model.add(BatchNormalization())
    self.model.add(Dropout(0.5))
    self.model.add(Dense(256, activation='relu'))
    self.model.add(BatchNormalization())
    self.model.add(Dropout(0.5))
    self.model.add(Dense(self.train_labels.shape[1] # length of classes
                         , activation='softmax'))
    self.model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    #model.summary() # output result

  # train the model for given epoches
  def fit(self,save_best_path,save_full_path):
    # prevent overfitting
    # ImageDataGenerator() randomly changes the characteristics of images and provides randomness in the data and makes dataset bigger?
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=180,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.3, # Randomly zoom image
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True, # randomly flip images
        )
    print('xtrain shape : {}'.format(self.x_train.shape))
    datagen.fit(self.x_train)
    # reduce the learning rate by 0.5
    lrr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=6, verbose=1, min_lr=0.00001)
    # save the best considered model with epoch number and validation accuracy
    checkpoints = ModelCheckpoint(save_best_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
    # save full model
    checkpoints_full = ModelCheckpoint(save_full_path, monitor='val_accuracy', verbose=1, save_best_only=False, mode='max')
    # train model with batch
    print("batch size:{}".format(self.x_train.shape[0]))
    # step_per_epoch * batch_size = len(train_data)
    train_model = self.model.fit_generator( datagen.flow(self.x_train,self.y_train,batch_size=self.batch_size)
                                 ,epochs=self.epoches
                                 ,validation_data=(self.x_val,self.y_val)
                                ,steps_per_epoch=self.x_train.shape[0]/self.batch_size # batch size
                                 #,validation_steps=len(self.x_val)
                                 ,callbacks=[checkpoints, lrr, checkpoints_full])
  # evalutate the model performance
  def evaluate_model(self):
    # load the best model
    self.model.load_weights('/content/drive/MyDrive/AI_final/weights.best_77-0.90.hdf5')
    # print evaluation
    print(self.model.evaluate(self.x_train,self.y_train))
    print(self.model.evaluate(self.x_val,self.y_val))
    # use confusion matrix to evaluate the prediction
    Y_true = np.argmax(self.y_val, axis=1) # ground truth
    Y_pred = self.model.predict(self.x_val) # prediction
    Y_pred_class = np.argmax(Y_pred, axis=1) # predict the class
    cmatrix = confusion_matrix(Y_true, Y_pred_class)
    print("\n confusion matrix:")
    print(cmatrix)
    

In [17]:
if __name__ == "__main__":
    train = Image_dataset()
    train.load_train_file('/content/drive/MyDrive/AI_final/train')
    train.clean_img()
    train.label_encoding()
    train.split_data()
    

Sugar beet: load 385 images done!
Small-flowered Cranesbill: load 496 images done!
Maize: load 221 images done!
Shepherds Purse: load 231 images done!
Loose Silky-bent: load 654 images done!
Fat Hen: load 475 images done!
Cleavers: load 287 images done!
Scentless Mayweed: load 516 images done!
Common wheat: load 221 images done!
Common Chickweed: load 611 images done!
Charlock: load 390 images done!
Black-grass: load 263 images done!


Orignal xtrain shape : (4750, 70, 70, 3)


Classes['Black-grass' 'Charlock' 'Cleavers' 'Common Chickweed' 'Common wheat'
 'Fat Hen' 'Loose Silky-bent' 'Maize' 'Scentless Mayweed'
 'Shepherds Purse' 'Small-flowered Cranesbill' 'Sugar beet']

---Split training and validation sets---
xtrain shape : (4275, 70, 70, 3)
ytrain shape : (4275, 12)
  xval shape : (475, 70, 70, 3)
  yval shape : (475, 12)
----------------------------------------------


In [18]:
    
    train2 = convolutional_network(epoches=80,batch_size=128)
    train2.network_layer()
    

In [ ]:
    # train for a long time!
    train2.fit("drive/MyDrive/AI_final/weights.best_{epoch:02d}-{val_accuracy:.2f}.hdf5","drive/MyDrive/AI_final/weights.last_auto4.hdf5")

In [19]:
    train2.evaluate_model()

134/134 [==============================] - 128s 946ms/step - loss: 0.2364 - accuracy: 0.9120
[0.23635384440422058, 0.9120467901229858]
15/15 [==============================] - 14s 934ms/step - loss: 0.2880 - accuracy: 0.9032
[0.2879680395126343, 0.9031578898429871]

 confusion matrix:
[[11  0  0  0  2  0 13  0  0  0  0  0]
 [ 0 39  0  0  0  0  0  0  0  0  0  0]
 [ 0  3 23  0  0  0  0  0  0  0  3  0]
 [ 0  0  0 57  0  0  0  1  2  0  0  1]
 [ 1  0  0  0 20  0  0  1  0  0  0  0]
 [ 1  0  0  0  0 44  1  1  0  0  0  0]
 [ 5  0  0  0  1  0 58  0  1  0  0  0]
 [ 0  0  0  0  0  0  0 22  0  0  0  0]
 [ 0  0  1  0  0  1  1  0 48  0  0  1]
 [ 0  0  0  0  0  0  0  0  2 21  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 50  0]
 [ 0  1  0  0  0  0  0  2  0  0  0 36]]


In [27]:
    test = Image_dataset()
    test.load_test_file('/content/drive/MyDrive/AI_final/test')

load 794 images done!


Test xtrain shape : (794, 70, 70, 3)


In [28]:
    test.clean_img()
    # predict the test files
    test.train_set = test.train_set/255 # normalization
    prediction = train2.model.predict(test.train_set)
    # prediction to csv file
    predict = np.argmax(prediction, axis=1)
    pred_label = train.original_labels.classes_[predict]
    result = {'file':test.train_labels, 'species':pred_label}
    result = pd.DataFrame(result)
    result.to_csv("/content/drive/MyDrive/AI_final/Prediction.csv", index=False)